**Równania różniczkowe - spectral bias**

In [11]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import deepxde as dde
# import tensorflow as tf
import deepxde.backend as tf
tf.set_default_backend("tensorflow.compat.v1")

# import tensorflow.compat.v1 as tf
# tf.disable_eager_execution()


NotImplementedError: Unsupported backend: tensorflow.compat.v2.
Please select backend from tensorflow.compat.v1, tensorflow, pytorch, jax or paddle.

**Zadanie 1.**

In [ ]:
# Problem specific data
a = -2*np.pi
b = 2*np.pi

In [ ]:
# Procedure definition
def Procedure(omega, hidden_layers, neurons_per_layer, num_training, num_test):
    # Domain
    def ode(x, y) :
        du_x = dde . grad . jacobian (y , x , i =0 , j =0)
        return du_x - tf.cos(omega*x)
    
    geom = dde.geometry.Interval(a, b)
    
    # Conditions and training data
    bc = dde.icbc.DirichletBC(geom, lambda x: 0,
                    lambda _, on_boundary: on_boundary)

    data = dde.data.PDE(
        geom,
        ode,
        bc,
        num_domain = num_training,
        num_test = num_test,
    )

    # Network
    layer_size = [1] + [neurons_per_layer]*hidden_layers + [1]
    activation = "tanh"
    initializer = "Glorot normal"
    net = dde.nn.FNN(layer_size, activation, initializer)

    # Model
    model = dde.Model(data, net)
    optimizer = "adam"
    model.compile(optimizer, lr=0.001)
    losshistory, train_state = model.train(iterations=50000)

    # Results
    x_data = np.linspace(a, b, num=num_test)[:, None]

    u_pred = model.predict(x_data)
    u_true = 1/omega * np.sin(omega*x_data)
    residual = model.predict(x_data, operator=ode)

    return x_data, u_pred, u_true, residual


In [ ]:
x_data, u_pred, u_true, residual = Procedure(1.0, 2, 16, 200, 1000)

# Plot
plt.title('PINN, analytical solutions comparison')
plt.plot(x_data, u_pred, label='model prediction for u')
plt.plot(x_data, u_true, label='analytical solution for u', linestyle='dashed')
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
        fancybox=True, shadow=True, ncol=1)
plt.grid()
plt.show()

In [ ]:
for layers, neurons in [(2, 16), (4, 64), (5, 128)]:  
        x_data, u_pred, u_true, residual = Procedure(15.0, layers, neurons, 200*15, 5000)

        # Plot
        plt.title('PINN, analytical solutions comparison')
        plt.plot(x_data, u_pred, label='model prediction for u')
        plt.plot(x_data, u_true, label='analytical solution for u', linestyle='dashed')
        plt.xlabel('x')
        plt.ylabel('u')
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                fancybox=True, shadow=True, ncol=1)
        plt.grid()
        plt.show()